<a href="https://colab.research.google.com/github/rafaelbuzi/Limpeza-de-Dados/blob/CAGED/Per_Capita.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Tratando os dados do CAGED**

O Cadastro Geral de Empregados e Desempregados (CAGED) foi criado como registro permanente de admissões e dispensa de empregados, sob o regime da Consolidação das Leis do Trabalho (CLT). É utilizado pelo Programa de Seguro-Desemprego, para conferir os dados referentes aos vínculos trabalhistas, além de outros programas sociais.

Os dados referente ao CAGED podem ser obtidos no link abaixo:

http://pdet.mte.gov.br/novo-caged

## **0. Import das Bibliotecas e carregamento dos dados**

In [1]:
from urllib import request
# Define the remote file to retrieve
remote_url = 'http://pdet.mte.gov.br/images/Novo_CAGED/Out2022/3-tabelas.xlsx'
# Define the local filename to save data
local_file = 'caged.xlsx'
# Download remote and save locally
request.urlretrieve(remote_url, local_file)

('caged.xlsx', <http.client.HTTPMessage at 0x7fa1adfec110>)

In [2]:
import pandas as pd
import numpy as np
df = pd.read_excel('caged.xlsx', sheet_name="Tabela 8.1")

/usr/local/lib/python3.7/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


## **1. Ajustes**

Preenchendo os valores horizontalmente: *horizontal forward fill (axis = 1*)

Cabe ressaltar que ainda assim, haverá valores vazios.

In [3]:
df.dropna(axis=0, how='all', inplace=True)
df.dropna(axis=1, how='all', inplace=True)

In [4]:
df = df.ffill(axis=1)

In [5]:
df.columns = (df.iloc[1] + '_' + df.iloc[2])

In [6]:
df = df.iloc[3:, 1:]

In [7]:
df.columns.values[0] = 'Código'
df.columns.values[1] = 'Município'

In [8]:
df = pd.melt(df, id_vars=['Código', 'Município'])

In [9]:
df = df[~(df["Código"].str.contains('\*') == True)]
df = df[~(df["Código"].str.contains('Fonte') == True)]
df = df[~(df["Código"].str.contains('Total') == True)]
df = df[~(df["variable"].str.contains('\*') == True)]
df = df[~(df["variable"].str.contains('%') == True)]
df = df[~(df["variable"].str.contains('Acumulado') == True)]

In [10]:
df[['Data', 'Tipo']] = df['variable'].str.split('_', expand=True)
df.drop('variable', axis=1, inplace=True)

In [11]:
df['Ano'] = df.Data.str.extract('/(.*)').copy()
df['Mês'] = df.Data.str.split('/').str[0].copy()

In [12]:
df['Data Completa'] = np.where(df.Mês=='Janeiro', '01/01/' + df['Ano'], 
                      np.where(df.Mês=='Fevereiro', '01/02/' + df['Ano'],
                      np.where(df.Mês=='Março', '01/03/' + df['Ano'],
                      np.where(df.Mês=='Abril', '01/04/' + df['Ano'],
                      np.where(df.Mês=='Maio', '01/05/' + df['Ano'],
                      np.where(df.Mês=='Junho', '01/06/' + df['Ano'],
                      np.where(df.Mês=='Julho', '01/07/' + df['Ano'],
                      np.where(df.Mês=='Agosto', '01/08/' + df['Ano'],
                      np.where(df.Mês=='Setembro', '01/09/' + df['Ano'],
                      np.where(df.Mês=='Outubro', '01/10/' + df['Ano'],
                      np.where(df.Mês=='Novembro', '01/11/' + df['Ano'],
                     np.where(df.Mês=='Dezembro', '01/12/' + df['Ano'],'a'))))))))))))             

In [13]:
df.drop(columns=['Data', 'Ano', 'Mês'], inplace=True)

In [14]:
df['Data Completa'] = pd.to_datetime(df['Data Completa'], dayfirst=True)

In [15]:
df['Valor'] = pd.to_numeric(df.value, errors='coerce')
df = df.fillna(0)

In [16]:
df2022 = df[df['Data Completa'].dt.year==2022]

In [17]:
df2022 = df2022.groupby(['Município', 'Código', 'Tipo'])['Valor'].sum().reset_index()
df2022['Código'] = df2022['Código'].astype('str')

Os dados populacionais para os cálculos per capita podem ser obtidos no link abaixo:

https://sidra.ibge.gov.br/tabela/6579

In [18]:
pop = pd.read_excel('tabela6579.xlsx')
pop = pop.iloc[3:]
pop.columns = ['Código', 'Município', 'População']
pop['Código'] = pop['Código'].str[:6]

In [19]:
df2022 = df2022.merge(pop[['Código', 'População']], 
                      left_on='Código', right_on='Código')

In [20]:
df2022 = df2022[df2022['Tipo']=='Saldos']

In [21]:
df2022['População'] = pd.to_numeric(df2022.População, errors='coerce')
df2022 = df2022.fillna(0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [22]:
df2022['Per Capita'] = (df2022.Valor / df2022.População) * 100000

In [23]:
df2022.head(2)

,Município,Código,Tipo,Valor,População,Per Capita
3,Ac-Acrelandia,120001,Saldos,-11.0,15721,-69.970104
7,Ac-Assis Brasil,120005,Saldos,34.0,7649,444.502549


In [24]:
df2022.reset_index(drop=True, inplace=True)

In [25]:
df2022.to_excel('df2022.xlsx')